In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("Reviews.csv")
sorted_data = dataset.sort_values('ProductId')
data = sorted_data.drop_duplicates(subset={'UserId','ProfileName','Time','Text'})
data[data['HelpfulnessNumerator'] > data['HelpfulnessDenominator']]
data = data[data['HelpfulnessNumerator'] <= data['HelpfulnessDenominator']]

In [3]:
data = data[data['Score'] != 3]
data['Score'] = data['Score'].replace([1,2],0)
data['Score'] = data['Score'].replace([4,5],1)
data = data.sort_values(by=['Time'])

In [4]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import nltk

In [5]:
Text=data['Text']
stop = set(stopwords.words('english'))
sno= nltk.stem.SnowballStemmer('english')
stop.remove('not')

In [6]:
#remove unneccessary symbols
postive_word = []
negative_word = []
final_text = []
j=0
s=''
for i in Text.values:
    sent = []
    i = re.sub(r'[^a-zA-Z]',' ',i)
    for words in i.split():
        if(len(words)>2) and words not in stop:
            #s=(sno.stem(words.lower()))
            s=words.lower()
            sent.append(s);
            if data['Score'].values[j] == 1:
                postive_word.append(s)
            if data['Score'].values[j] == 0:
                negative_word.append(s)
    final_text.append(sent)
    j += 1
            
  

In [7]:

len(final_text)

364159

In [8]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

C:\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
glov_file = 'glove.6B.100d.txt'
op_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glov_file,op_file)
model = KeyedVectors.load_word2vec_format(op_file)


# avg w2vec

In [10]:
avg_text_vec = []
for sen in final_text:
    sen_vec = np.zeros(100)
    count=0
    for word in sen:
        try:
            vec = model.wv[word]
            sen_vec += vec;
            count += 1
        except:
            pass
    sen_vec /= count
    avg_text_vec.append(sen_vec)

print(len(avg_text_vec))
print(len(avg_text_vec[0]))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


364159
100


In [11]:
avg_text_vec_np = pd.DataFrame(avg_text_vec)


In [12]:
op_data = data['Score'].values

In [13]:
data['Score'].value_counts()

1    307052
0     57107
Name: Score, dtype: int64

In [14]:
for i,k in enumerate(avg_text_vec_np[0].isnull()):
    if(k):
        print(i)

192734
227581
356684


In [15]:
avg_text_vec_np=avg_text_vec_np.dropna(axis=0)
op_data=np.delete(op_data,[192734,227581,356684])
avg_text_vec_np = avg_text_vec_np.values

In [16]:
train_in = avg_text_vec_np[:290000,:]
test_in = avg_text_vec_np[290000:,:]
train_op = op_data[:290000]
test_op = op_data[290000:]

# applying models

# knn

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

In [18]:
for k in range(1,20):
    knn = KNeighborsClassifier(n_neighbors=k)
    model = knn.fit(train_in[:1000,:],train_op[:1000])
    pred = model.predict(test_in[:1000,:])
    acc = f1_score(test_op[:1000],pred)
    print(str(k) + "  "+ str(acc))

1  0.8700696055684456
2  0.856628674265147
3  0.8945290468133108
4  0.8897772701313535
5  0.8939478067740144
6  0.8935933147632311
7  0.893687707641196
8  0.8929561841375485
9  0.893687707641196
10  0.8945615982241953
11  0.8941828254847646
12  0.893569844789357
13  0.8947951273532669
14  0.8941828254847646
15  0.8941828254847646
16  0.8941828254847646
17  0.8947951273532669
18  0.8947951273532669
19  0.8947951273532669


In [19]:
knn = KNeighborsClassifier(n_neighbors=7)
model = knn.fit(train_in,train_op)



In [20]:
pred = model.predict(test_in[:10000])

In [21]:
acc = f1_score(test_op[:10000],pred)

In [22]:
print(acc)

0.9165434906196702


# naive bayes

In [23]:
from sklearn.naive_bayes import GaussianNB

In [24]:
nb = GaussianNB()
model_nb = nb.fit(train_in,train_op)
pred_nb = model_nb.predict(test_in)

In [25]:
acc_nb = f1_score(test_op,pred_nb)
print(acc_nb)

0.8187759205730804


# logistic regerssion

In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
model_lr = LogisticRegression().fit(train_in,train_op)
pred_lr = model_lr.predict(test_in)

In [28]:
acc_lr = f1_score(test_op,pred_lr)
print(acc_lr)

0.9251522860410297


# SVM

In [29]:
from sklearn.svm import SVC

In [30]:
model_svm = SVC().fit(train_in[:10000,:],train_op[:10000])
pred_svm = model_svm.predict(test_in[:10000,:])

In [31]:
acc_svm= f1_score(test_op[:10000],pred_svm)
print(acc_svm)

0.905488972801401


# decision tree

In [32]:
from sklearn.tree import DecisionTreeClassifier

In [33]:
model_dt = DecisionTreeClassifier(random_state=0).fit(train_in,train_op)
pred_dt = model_dt.predict(test_in)

In [34]:
acc_dt= f1_score(test_op,pred_dt)
print(acc_dt)

0.8647169348909403


# random forest

In [35]:
from sklearn.ensemble import RandomForestClassifier

C:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [36]:
model_rf = RandomForestClassifier(random_state=0).fit(train_in,train_op)
pred_rf = model_rf.predict(test_in)

In [37]:
acc_rf= f1_score(test_op,pred_rf)
print(acc_rf)

0.9094091225896384


# boosting classifiers

In [38]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
model_gb = GradientBoostingClassifier(random_state=0).fit(train_in,train_op)
pred_gb = model_gb.predict(test_in)

In [ ]:
acc_gb= f1_score(test_op,pred_gb)
print(acc_gb)

## adaboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
model_ab =AdaBoostClassifier(random_state=0).fit(train_in,train_op)
pred_ab = model_ab.predict(test_in)

In [ ]:
acc_ab= f1_score(test_op,pred_ab)
print(acc_ab)